In [2]:
from utils.tokenizer import load_tokenizer
import pandas as pd
import os
from utils.const import PREPROCESSED_DATA_DIR
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, metrics
import tensorflow as tf

2023-01-07 19:27:29.086821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def load_datasets() -> tuple[pd.DataFrame]:
    return (
        pd.read_excel(os.path.join(PREPROCESSED_DATA_DIR, "train.xlsx")),
        pd.read_excel(os.path.join(PREPROCESSED_DATA_DIR, "valid.xlsx")),
        pd.read_excel(os.path.join(PREPROCESSED_DATA_DIR, "test.xlsx")),
    )

### Tokenization

In [4]:
trn, val, test = load_datasets()

tokenizer = load_tokenizer()
tokens_trn = tokenizer.texts_to_sequences(trn["text"])
tokens_val = tokenizer.texts_to_sequences(val["text"])
tokens_test = tokenizer.texts_to_sequences(test["text"])

# T = (trn["text"].map(lambda x: len(x)).max())  # Input sequence size / max sequence length
T = 20  # Input sequence size / max sequence length

# Padding <- kazdy tekst ma te sama dlugosc
padding_trn = tf.keras.utils.pad_sequences(tokens_trn, T, padding="post")
padding_val = tf.keras.utils.pad_sequences(tokens_val, T, padding="post")
padding_test = tf.keras.utils.pad_sequences(tokens_val, T, padding="post")

In [5]:
for emotion in trn.columns[1:3]:
    y_trn = trn[emotion]
    # y_val = val[emotion]
    y_test = test[emotion]

    # train
    clf = svm.SVC(kernel="linear")
    clf.fit(padding_trn, y_trn)

    # test
    y_pred = clf.predict(tokens_test)

    print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))